# Task 5 Metrics & Validation
Analyze trained attention CGAN outputs with FID (optional), CLIP score (placeholder), and conditional accuracy.

In [7]:
%pip install -q torch-fidelity open-clip-torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import torch, os, numpy as np, json
from pathlib import Path
from torch_fidelity import calculate_metrics
import sys, importlib, importlib.util, types, inspect

# Robust path + explicit module resolution for task4.gans
CWD = Path.cwd().resolve()
POSSIBLE_ROOTS = [CWD, CWD.parent]
# Insert roots (not their parents' task4) early so package discovery prefers workspace copy
for r in POSSIBLE_ROOTS:
    if str(r) not in sys.path:
        sys.path.insert(0, str(r))

loaded = False
module = None
first_error = None
try:
    module = importlib.import_module('task4.gans')
except Exception as e:  # capture but keep trying explicit file load
    first_error = e

# If module imported but lacks AttnGenerator, or import failed, try explicit file targeting the version that defines class AttnGenerator
need_explicit = module is None or not hasattr(module, 'AttnGenerator')
if need_explicit:
    for root in POSSIBLE_ROOTS:
        gans_path = root / 'task4' / 'gans.py'
        try:
            if gans_path.exists() and 'class AttnGenerator' in gans_path.read_text(encoding='utf-8', errors='ignore'):
                spec = importlib.util.spec_from_file_location('task4.gans', gans_path)
                mod = importlib.util.module_from_spec(spec)
                spec.loader.exec_module(mod)  # type: ignore
                sys.modules['task4.gans'] = mod
                module = mod
                break
        except Exception as ee:
            first_error = first_error or ee

if module is None or not hasattr(module, 'AttnGenerator'):
    snippet = sys.path[:6]
    raise ImportError(f"Could not load AttnGenerator from task4.gans. First error={first_error}. sys.path head={snippet}")

# Extract required symbols
AttnGenerator = module.AttnGenerator
LATENT_DIM = module.LATENT_DIM
conditional_accuracy = module.conditional_accuracy
_build_or_load_mnist_classifier = module._build_or_load_mnist_classifier

print('Using task4.gans from:', inspect.getfile(module))

from torchvision.utils import save_image
DEVICE=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
OUT='task5_outputs'
G=AttnGenerator().to(DEVICE)
w=Path(OUT)/'attn_generator.pt'
assert w.exists(), 'Train Task5 model first and ensure attn_generator.pt exists in task5_outputs/'
G.load_state_dict(torch.load(w, map_location=DEVICE)); G.eval()

Using task4.gans from: D:\acm\models\jul25\nullclass\task4\gans.py


AttnGenerator(
  (label_emb): Embedding(10, 50)
  (project): Sequential(
    (0): Linear(in_features=150, out_features=6272, bias=True)
    (1): BatchNorm1d(6272, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (net): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): SelfAttention2d(
      (f): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1))
      (g): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1))
      (h): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
      (v): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (4): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(ne

## 1. Generate Sample Set for Metrics

In [9]:
sample_dir=Path('task5_samples'); sample_dir.mkdir(exist_ok=True)
with torch.no_grad():
    for i in range(500):
        lbl=torch.randint(0,10,(1,),device=DEVICE)
        z=torch.randn(1, LATENT_DIM, device=DEVICE)
        img=G(z,lbl)*0.5+0.5
        save_image(img, sample_dir/f'{i}.png')
len(list(sample_dir.iterdir()))

500

## 2. FID vs Real MNIST (subset)

In [11]:
from torchvision import datasets, transforms
real_dir=Path('task5_real'); real_dir.mkdir(exist_ok=True)
if not any(real_dir.iterdir()):
    tf=transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5],[0.5])])
    ds=datasets.MNIST(root='./data', train=True, download=True, transform=tf)
    for i,(img,_) in enumerate(torch.utils.data.DataLoader(ds,batch_size=1,shuffle=True)):
        if i>=500: break
        save_image(img*0.5+0.5, real_dir/f'{i}.png')
metrics = calculate_metrics(input1=str(sample_dir), input2=str(real_dir), fid=True, kid=False, pr=False, verbose=False, cuda=False)
metrics

d:\acm\models\jul25\nullclass\.venv\Lib\site-packages\torch_fidelity\datasets.py:16: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(img.tobytes())).view(height, width, 3)


{'frechet_inception_distance': 60.570133823208806}

## 3. Conditional Accuracy

In [12]:
clf=_build_or_load_mnist_classifier(DEVICE)
cond_metrics=conditional_accuracy(G, clf, DEVICE)
cond_metrics

{'accuracy': np.float64(0.806),
 'precision': 0.8791856357914174,
 'recall': 0.806,
 'f1': 0.7936060303473031}

## 4. Save All Metrics

In [13]:
all_metrics={'fid':metrics.get('frechet_inception_distance'), **cond_metrics}
json.dump(all_metrics, open('task5_metrics.json','w'), indent=2); all_metrics

{'fid': 60.570133823208806,
 'accuracy': np.float64(0.806),
 'precision': 0.8791856357914174,
 'recall': 0.806,
 'f1': 0.7936060303473031}

## 5. Summary
Task 5 metrics computed and stored in task5_metrics.json.